<a href="https://colab.research.google.com/github/chandlereskey17/phys341/blob/master/Exam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import scipy.stats as stats
import numpy as np

df = 5
print ("%0.2f" % (stats.chi2.ppf(0.90,df)))

df = 12
print ("%0.2f" % (stats.chi2.ppf(0.01,df)))

df = 23
print ("%0.3f" % (stats.chi2.cdf(32.01,df)-stats.chi2.cdf(10.20,df)))

df = 23
print ("%0.3f" % (1-(stats.chi2.cdf(41.638,df)-stats.chi2.cdf(10.196,df))))

9.24
3.57
0.890
0.020


In [4]:
n = 9
df = n - 1
s = 2.87
cl = 0.95

c_interval = stats.chi2.interval(cl,df)

print ("Chi2 Interval: (%0.2f, %0.2f)" % c_interval)

var_high = df*s**2/c_interval[0]
var_low = df*s**2/c_interval[1]

print ("Variance Interval: (%0.2f, %0.2f)" % (var_low,var_high))
print ("Std. Deviation Interval: (%0.2f, %0.2f)" % (np.sqrt(var_low),np.sqrt(var_high)))

Chi2 Interval: (2.18, 17.53)
Variance Interval: (3.76, 30.23)
Std. Deviation Interval: (1.94, 5.50)


In [9]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

def fpvalue(fvalue,dof1,dof2,test):
    fdist = stats.f(dof1,dof2)

    if (fvalue > 1):
        if test == "upper":
            pvalue = (1-fdist.cdf(fvalue))
        if test == "two":
            pvalue = 2.0*(1-fdist.cdf(fvalue))
        if test == "lower":
            pvalue = fdist.cdf(fvalue)
    else:
        if test == "upper":
            pvalue = fdist.cdf(fvalue)
        if test == "two":
            pvalue = 2.0*fdist.cdf(fvalue)
        if test == "lower":
            pvalue = (1-fdist.cdf(fvalue))
            
    print ("Pvalue = %0.3f" % (pvalue))

yf = np.array([	33,	27,	29,	27,	32,	36,	27,	27,	26,	31])
of = np.array([14,	12,	20,	13,	16])

n1 = len(yf)
n2 = len(of)

dof1 = n1-1
dof2 = n2-1

s1 = yf.std(ddof=1)
s2 = of.std(ddof=1)

xbar1 = yf.mean()
xbar2 = of.mean()

fvalue = s1**2/s2**2

print ("Standard Deviations of the two samples = %.1f, %.1f" %(s1,s2))
print ("Variances of the two samples = %.1f, %.1f" %(s1**2,s2**2))
print ("F statistic = %0.3f" % fvalue)

alpha = 0.05

fdist = stats.f(dof1,dof2)
flow = fdist.ppf(alpha/2)
fhigh = fdist.ppf(1-alpha/2)

print ("Critical F-values = %0.3f, %0.3f" % (flow,fhigh))

if (fvalue > 1):
    pvalue = 2.0*(1-fdist.cdf(fvalue))
else:
    pvalue = 2.0*fdist.cdf(fvalue)
    
print ("Pvalue = %0.3f" % (pvalue))

pvalue = fpvalue(fvalue,dof1,dof2,"two")

Standard Deviations of the two samples = 3.3, 3.2
Variances of the two samples = 11.2, 10.0
F statistic = 1.117
Critical F-values = 0.212, 8.905
Pvalue = 0.990
Pvalue = 0.990


In [11]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

a = 4
n = 8

# calculate DOF's
N = a*n
dof_total = N - 1
dof_treatment = a - 1
dof_error = dof_total - dof_treatment
print (dof_error)

# summary data for xbar
xbar = np.array([4.39,4.52,5.49,6.36])
xbarbar = xbar.mean()

# calculate SS_treatment
SS_treatment = 0.0
for x in xbar:
    SS_treatment += n*(x-xbarbar)**2
    
MS_treatment = SS_treatment/dof_treatment

sumxij2 = 911.91

SS_total = sumxij2 - N*xbarbar**2
    
SS_error = SS_total - SS_treatment
MS_error = SS_error/dof_error

fSN = MS_treatment/MS_error

print ("F Signal/Noise = %0.2f " % fSN)

alpha = 0.05

fdist = stats.f(dof_treatment,dof_error)
fhigh = fdist.ppf(1-alpha)

print ("Critical F-value = %0.2f" % (fhigh))

if (fSN > 1):
    pvalue = (1-fdist.cdf(fSN))
else:
    pvalue = fdist.cdf(fSN)
    
print ("Pvalue = %0.3f" % (pvalue))








# Tukey's Procedure
#
# Step 1:  Calculate the standard error = sqrt(MS_error/n)

from q_tukey import q_tukey

std_error = np.sqrt(MS_error/n)
print ("Standard Error = %0.3f" % std_error)

# Step 2:  Order the means from smallest to largest

print ("Original Xbar = ",xbar)
xbar_orig = xbar.copy()
xbar.sort()
print ("Sorted Xbar = ",xbar)

# Step 3:  Get the expected number of error bars (sem) between largest and smallest, for the number
# of means to be compared (a).  For this, we need the studentized range table values q(alpha,dof_error,k) for k=2..a.
# 
# http://www.real-statistics.com/statistics-tables/studentized-range-q-table/
# For this problem, dof_error = 28, alpha = 0.05, k = 2,3,4
# q = np.array([2.897,3.499,3.861])

n_means=len(xbar)
q = q_tukey(n_means,dof_error,alpha)
print (q)

# Step 4:  Calculate W = q(k=5)*std_error

W = q[n_means-2]*std_error

print ("W = %0.1f" % W)

# Step 5:  Make pair-wise comparisons

print (list(xbar_orig))

print ("i j xbar1 xbar2 Diff Diff_comp Result")
for i in range(len(xbar)):
    for j in range(len(xbar)):
        if (xbar[j] > xbar[i]):
            diff = (xbar[j] - xbar[i])/std_error
            diff_comp = q[j-i-1]
            if (diff>diff_comp):
                Result = "Yes - Difference"
            else:
                Result = "No Difference"
            iorig = list(xbar_orig).index(xbar[i])+1
            jorig = list(xbar_orig).index(xbar[j])+1
            print ("%0.0f %0.0f %0.2f %0.2f %0.3f  %0.3f %s" % (iorig,jorig,xbar[i],xbar[j],diff,diff_comp,Result))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


28
F Signal/Noise = 6.43 
Critical F-value = 2.95
Pvalue = 0.002


ModuleNotFoundError: ignored